In [4]:
import pandas as pd
import numpy as np
import requests
from datetime import date, timedelta
from scipy.optimize import newton
import urllib3

# Silenciar avisos de seguridad
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- 1. FUNCIÓN DE SCRAPING (Tu función optimizada) ---
def obtener_tabla(url):
    try:
        # Usamos flavor='html5lib' para mayor robustez
        tablas = pd.read_html(url, thousands='.', decimal=',', flavor='html5lib')
        if not tablas: return None
        
        tabla = tablas[0]
        mapa_columnas = {
            'Símbolo': 'Nombre', 'Ticker': 'Nombre', 
            'Último Operado': 'PRECIO', 'Último': 'PRECIO'
        }
        
        columnas_a_renombrar = {orig: nuevo for orig, nuevo in mapa_columnas.items() if orig in tabla.columns}
        tabla = tabla.rename(columns=columnas_a_renombrar)
        
        if 'PRECIO' in tabla.columns:
            tabla['PRECIO'] = tabla['PRECIO'].astype(str).str.replace(r'\.(?=\d{3}(?:,|$))', '', regex=True)
            tabla['PRECIO'] = tabla['PRECIO'].str.replace(',', '.', regex=False).str.strip()
            tabla['PRECIO'] = pd.to_numeric(tabla['PRECIO'], errors='coerce')
        
        if 'Nombre' in tabla.columns:
            tabla["TKR"] = tabla["Nombre"].astype(str).str.extract(r"^([A-Z0-9.]+)")
            
        return tabla
    except Exception as e:
        print(f"Error en scraping: {e}")
        return None

# --- 2. FUNCIÓN BCRA ---
def obtener_ultimo_cer():
    # Pedimos un rango de 10 días hacia atrás para asegurarnos de encontrar algo
    hasta = date.today().isoformat()
    desde = (date.today() - timedelta(days=10)).isoformat()
    
    url = "https://api.bcra.gob.ar/estadisticas/v4.0/monetarias/30"
    
    try:
        r = requests.get(url, params={'desde': desde, 'hasta': hasta}, verify=False, timeout=10)
        
        if r.status_code == 200:
            data = r.json()
            if 'results' in data and data['results']:
                detalle = data['results'][0].get('detalle', [])
                
                if detalle:
                    # Ordenamos la lista: el valor con la fecha más reciente queda en la posición [0]
                    # Convertimos la fecha a string o datetime para comparar correctamente
                    detalle_ordenado = sorted(detalle, key=lambda x: x['fecha'], reverse=True)
                    
                    ultimo_registro = detalle_ordenado[0]
                    return float(ultimo_registro['valor'])
                
            print("⚠️ No se encontraron datos en el detalle del BCRA.")
        else:
            print(f"❌ Error de API: Código {r.status_code}")
            
    except Exception as e:
        print(f"❌ Error de conexión al BCRA: {e}")
        
    return None



# --- 3. CLASE BONO CER ---
class BonoCER:
    def __init__(self, nombre, tasa_anual, cuotas_amort, inicio_amort, fin_amort, cer_emision):
        self.nombre = nombre
        self.tasa_anual = tasa_anual
        self.cuotas_amort = cuotas_amort
        self.inicio_amort = inicio_amort
        self.fin_amort = fin_amort
        self.cer_emision = cer_emision

    def obtener_flujos(self, hoy):
        anio, mes = self.inicio_amort.year, self.inicio_amort.month
        valor_cuota_capital = 100 / self.cuotas_amort
        capital_residual = 100.0
        capital_residual_hoy = 100.0
        
        fechas_pago, montos_flujo = [], []
        fecha_pago_actual = date(anio, mes, 9)
        
        while fecha_pago_actual <= self.fin_amort:
            interes = capital_residual * (self.tasa_anual / 2)
            if fecha_pago_actual > hoy:
                fechas_pago.append(fecha_pago_actual)
                montos_flujo.append(interes + valor_cuota_capital)
            else:
                capital_residual_hoy -= valor_cuota_capital
            
            capital_residual -= valor_cuota_capital
            if mes == 5: mes = 11
            else: 
                mes = 5
                anio += 1
            fecha_pago_actual = date(anio, mes, 9)
        return fechas_pago, montos_flujo, capital_residual_hoy

    def calcular_tir(self, precio_sucio, cer_hoy):
        hoy = date.today()
        inversion_real = precio_sucio / (cer_hoy / self.cer_emision)
        fechas_f, montos_f, residual = self.obtener_flujos(hoy)
        
        fechas_np = np.array([hoy] + fechas_f)
        valores_np = np.array([-inversion_real] + montos_f)
        
        def vpn(tasa):
            anios = np.array([(f - hoy).days / 365.0 for f in fechas_np])
            return np.sum(valores_np / ((1 + tasa) ** anios))
        
        try:
            return newton(vpn, 0.1), residual
        except:
            return None, residual

# --- 4. CÓDIGO DE EXTRACCIÓN Y CÁLCULO ---

# Descargamos el DataFrame UNA SOLA VEZ
url_iol = 'https://iol.invertironline.com/mercado/cotizaciones/argentina/bonos/todos'
df_mercado = obtener_tabla(url_iol)
cer_actual = obtener_ultimo_cer()

# Definimos los objetos
tx26 = BonoCER("TX26", 0.02, 5, date(2024, 11, 9), date(2026, 11, 9), 22.5440)
tx28 = BonoCER("TX28", 0.0225, 10, date(2024, 5, 9), date(2028, 11, 9), 22.5440)



In [6]:
if df_mercado is not None and cer_actual:
    print(f"Valor del CER Hoy: {cer_actual}\n")
    
    for bono in [tx26, tx28]:
        # --- AQUÍ ESTÁ EL CÓDIGO PARA EXTRAER EL VALOR DEL DATAFRAME ---
        # Buscamos la fila donde el TKR coincida con el nombre del bono
        fila_bono = df_mercado.loc[df_mercado['TKR'] == bono.nombre]
        
        if not fila_bono.empty:
            # Extraemos el PRECIO como float usando .item()
            precio = fila_bono['PRECIO'].item()
            
            tir, res = bono.calcular_tir(precio, cer_actual)
            print(f"Bono: {bono.nombre} | Precio: ${precio:.2f}")
            print(f"  > Residual: {res:.2f}% | TIR Real: {tir*100:.2f}%")
            print("-" * 40)
        else:
            print(f"No se encontró cotización para {bono.nombre}")

Valor del CER Hoy: 670.3668

Bono: TX26 | Precio: $1167.00
  > Residual: 40.00% | TIR Real: 5.49%
----------------------------------------
Bono: TX28 | Precio: $1665.00
  > Residual: 60.00% | TIR Real: 6.97%
----------------------------------------
